<a href="https://colab.research.google.com/github/coneill000/French-Flashcards-with-Anki/blob/main/French_Speech_Recognition_and_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# French Speech Recognition and Translation
Welcome to my speech recognition and translation program. This was designed to speed up the process of making French Anki flashcards by using the audio captured from the textbook. The program will first recognize the French word being spoken and rename the audio file accordingly. It will then use that word recognition to translate the meaning into English. All of this information will be stored in a Google Sheets, where you can manually check any errors, missing vocab words, etc. Yeah it's kind of overly complicated but who cares. 

## Imports
Run this first so you have the necessary imports. The imports are split into two cells, one for pip installs and one for library imports

In [ ]:
!pip install pydub
!pip install SpeechRecognition
!pip3 install googletrans

     |████████████████████████████████| 32.8MB 122kB/s 
     |████████████████████████████████| 61kB 5.0MB/s 
     |████████████████████████████████| 972kB 13.4MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 102kB 10.6MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-cp36-none-any.whl size=15736 sha256=c78c487db1ecde30bffef60fe01e9e5d7da8d008145b55fd9051f6601c25ca7e
  Stored in directory: /root/.cache/pip/wheels/28/1a/a7/eaf4d7a3417a0c65796c547cff4deb6d79c7d14c2abd29273e
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=d9abe1f5deed2aa78dbd51dcbad9f0caf7dc83fff35d19de11b17c84e281722f
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built googletrans contextvars


In [ ]:
import os
import re
import gspread
import speech_recognition as sr
from pydub import AudioSegment
from googletrans import Translator, constants
from google.colab import auth
from oauth2client.client import GoogleCredentials

## Google Authorization
Run this to give the program access to your Google Sheets. Please make sure that you mount your Google Drive using Google Colab. 

In [ ]:
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

## User-Set Variables


In [ ]:
directory = '/content/drive/My Drive/Ch2Vocab' #change this to directory where audio files are stored
sheetname = 'En avant Ch2Vocab' #change this to desired name of Google Sheets document
tag = 'EnAvantCh2'

## Main Program (aka Under the Hood)

In [ ]:
successpath = os.path.join(directory, 'success') 
errorpath = os.path.join(directory, 'error')
os.mkdir(successpath)
os.mkdir(errorpath)

In [ ]:
r = sr.Recognizer()
translator = Translator()
frenchlist = []

In [ ]:
for filename in os.listdir(directory):
    is_match = bool(re.match("S.mp3|S-\d.mp3|S-\d\d.mp3|S-\d\d\d.mp3", filename))
    if is_match:
        src = f'{directory}/{filename}'
        dst = f'{directory}/{filename[:-4]}.wav'
        sound = AudioSegment.from_mp3(src)
        sound.export(dst, format="wav")
        temp = sr.AudioFile(dst)
        with temp as source:
            audio = r.record(source)
        try:
            french = r.recognize_google(audio, language='fr-FR')
            os.rename(src, f'{successpath}/{french}.mp3')
            frenchlist.append(french)
        except:
            os.rename(src, f'{errorpath}/{filename}')

In [ ]:
#sets up sheet/opens pre-existing sheet for info storage
sh = gc.create(sheetname)
worksheet = gc.open(sheetname).sheet1

In [ ]:
#sets up basic worksheet information
worksheet.update_cell(1, 1, "French")
worksheet.update_cell(1, 2, "English")
worksheet.update_cell(1, 3, "Audio")
worksheet.update_cell(1, 4, "Tags")

num = len(frenchlist) + 1
french_cells = worksheet.range(f'A2:A{num}')
meaning_cells = worksheet.range(f'B2:B{num}')
audio_cells = worksheet.range(f'C2:C{num}')
tag_cells = worksheet.range(f'D2:D{num}')

In [ ]:
#fills in Google Sheet with information
for i, cell in enumerate(french_cells):
    cell.value = frenchlist[i]

for i, cell in enumerate(meaning_cells):
    translation = translator.translate(frenchlist[i], src="fr")
    cell.value = translation.text

for i, cell in enumerate(audio_cells):
    cell.value = f"[sound:{frenchlist[i]}.mp3]"

for i, cell in enumerate(tag_cells):
    cell.value = tag

worksheet.update_cells(french_cells)
worksheet.update_cells(meaning_cells)
worksheet.update_cells(audio_cells)
worksheet.update_cells(tag_cells)

{'spreadsheetId': '1ndHSdxtLKCH_txTcJjCKmwqZ22KMjZyW1oNacYnKGL8',
 'updatedCells': 155,
 'updatedColumns': 1,
 'updatedRange': 'Sheet1!D2:D156',
 'updatedRows': 155}